In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
try:
    address = "/content/drive/My Drive/Doctor/Research/CNN/Colab/"
    data_csv = pd.read_csv(address+'Data_Ny.csv', index_col=0)
except:
    address = ""
    data_csv = pd.read_csv(address+'Data_Ny.csv', index_col=0)

In [ ]:
# Space-Time Convolutional Neural Network (STCNN)

M = 18 # The number of past data for input
H = 6 # The number of future data for output
N = 67 # The number of sites

in_size       = (N, M)
out_size      = N*H
concat_size   = int(N/2**3)*int(M/2**3) # 3 Max Pooling Layers
learning_rate = 1e-4
total_epoch   = 150

class STCNN(nn.Module):
    def __init__(self):
        super(STCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 128, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(128)
        self.conv2 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.fc    = nn.Linear(concat_size*32, out_size)

        self.optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.bn1(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.bn2(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, concat_size*32)
        x = self.fc(x)
        return x

    def train_net(self, x, y):
        x, y = torch.tensor(x,dtype=torch.float), torch.tensor(y,dtype=torch.float)
        loss = F.mse_loss(self.forward(x), y)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [ ]:
# STCNN training

data = np.array(data_csv)
data_size = np.size(data, 1)

train_data_size  = int(data_size*0.6)
train_batch_size = train_data_size - M - H + 1

batch_size = 128
total_batch = int((train_batch_size-1)/batch_size) + 1

train_input  = np.zeros((train_batch_size, 1, N, M))
train_output = np.zeros((train_batch_size, N*H))
for i in range(train_batch_size):
    train_input[i,:]  = np.reshape(data[:,i:i+M], (1, N, M))
    train_output[i,:] = np.reshape(data[:,i+M:i+M+H], (N*H))


val_data_size    = int(data_size*0.8)
val_batch_size   = val_data_size - train_data_size - M - H + 1

val_input  = np.zeros((val_batch_size, 1, N, M))
val_output = np.zeros((val_batch_size, N*H))
for i in range(val_batch_size):
    val_input[i,:]  = np.reshape(data[:,train_data_size+i:train_data_size+i+M], (1, N, M))
    val_output[i,:] = np.reshape(data[:,train_data_size+i+M:train_data_size+i+M+H], (N*H))


model = STCNN()
mse_train, mse_val = [], [] # Mean Squared Error
mae_train, mae_val = [], [] # Mean Absolute Error
for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_x = train_input[batch_size*i:batch_size*(i+1)]
        batch_y = train_output[batch_size*i:batch_size*(i+1)]
        model.train_net(batch_x, batch_y)
    
    train_predict = model.forward(torch.tensor(train_input, dtype=torch.float)).detach().numpy()
    mse_train = np.mean(np.square(train_predict - train_output))
    mae_train = np.mean(np.abs(train_predict - train_output))
    scale_train = np.mean(np.abs(train_output[1:] - train_output[:-1]))
    
    val_predict = model.forward(torch.tensor(val_input, dtype=torch.float)).detach().numpy()
    mse_val = np.mean(np.square(val_predict - val_output))
    mae_val = np.mean(np.abs(val_predict - val_output))
    scale_val = np.mean(np.abs(val_output[1:] - val_output[:-1]))

    NRMSE_train = round(100*np.sqrt(mse_train),2)
    MAPE_train  = round(100*mae_train,2)
    MASE_train  = round(mae_train/scale_train,2)
    NRMSE_val   = round(100*np.sqrt(mse_val),2)
    MAPE_val    = round(100*mae_val,2)
    MASE_val    = round(mae_val/scale_val,2)

    print("epoch: {}".format(epoch+1))
    print("NRMSE_train: {}%".format(NRMSE_train).ljust(25), end="")
    print("MAPE_train: {}%".format(MAPE_train).ljust(25), end="")
    print("MASE_train: {}".format(MASE_train).ljust(25))
    print("NRMSE_val: {}%".format(NRMSE_val).ljust(25), end="")
    print("MAPE_val: {}%".format(MAPE_val).ljust(25), end="")
    print("MASE_val: {}".format(MASE_val).ljust(25))
    print("------------------------------------------------------------------------------------------")

In [ ]:
# STCNN test

test_batch_size   = data_size - val_data_size - M - H + 1

test_input  = np.zeros((test_batch_size, 1, N, M))
test_output = np.zeros((test_batch_size, N, H))
for i in range(test_batch_size):
    test_input[i,:]  = np.reshape(data[:,val_data_size+i:val_data_size+i+M], (1, N, M))
    test_output[i,:] = np.reshape(data[:,val_data_size+i+M:val_data_size+i+M+H], (N, H))

test_predict = model.forward(torch.tensor(test_input, dtype=torch.float)).detach().numpy()
test_predict = np.reshape(test_predict, (-1, N, H))
mse_test = np.mean(np.square(test_predict - test_output))
mae_test = np.mean(np.abs(test_predict - test_output))
scale_test = np.mean(np.abs(test_output[1:] - test_output[:-1]))

NRMSE_test = round(100*np.sqrt(mse_test),2)
MAPE_test  = round(100*mae_test,2)
MASE_test  = round(mae_test/scale_test,2)

print("NRMSE_test: {}%".format(NRMSE_test).ljust(25), end="")
print("MAPE_test: {}%".format(MAPE_test).ljust(25), end="")
print("MASE_test: {}".format(MASE_test).ljust(25))

test_output_a = np.mean(test_output, axis=1)
test_predict_a = np.mean(test_predict, axis=1)
mae_test_a = np.mean(np.abs(test_predict_a - test_output_a))
scale_test_a = np.mean(np.abs(test_output_a[1:] - test_output_a[:-1]))

MAPE_test_a      = round(100*mae_test_a,2)
MAPE_improvement = round(100*(MAPE_test-MAPE_test_a)/MAPE_test,2)

print("MAPE_test (Aggregation): {}%".format(MAPE_test_a).ljust(36), end="")
print("MAPE_test Improvement: {}%".format(MAPE_improvement).ljust(36))